# H1B Labor Condition Applications (Form ETA-9035) 

In [1]:
# H1B Labor Condition Applications (Form ETA-9035)

# http://econdataus.com/h1bdata.htm

#source: US Department of Labor
#https://www.dol.gov/agencies/eta/foreign-labor/performance

#source: US Bureau of Labor Statistics 
#https://www.bls.gov/oes/tables.htm

# metadata: lca_cols
# https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Record_Layout_FY2022_Q3.pdf

# DOL Guidance on Determining OES Wage Levels
# https://www.aila.org/infonet/dol-guidance-on-determining-oes-wage-levels

# 
# https://www.dol.gov/sites/doPrevailing Wage Determination Policy Guidancelgov/files/ETA/oflc/pdfs/Policy_Nonag_Progs.pdf

# git pull https://github.com/JohnBroberg/H1B_LCA.git

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## global variables

In [2]:

#columns to be read from LCA Excel file
lca_cols = ['CASE_NUMBER','CASE_STATUS', 'RECEIVED_DATE', 'DECISION_DATE', 'ORIGINAL_CERT_DATE'
           , 'VISA_CLASS', 'SOC_CODE', 'SOC_TITLE', 'FULL_TIME_POSITION'
           , 'TOTAL_WORKER_POSITIONS', 'EMPLOYER_NAME', 'NAICS_CODE', 'WORKSITE_WORKERS'
           , 'SECONDARY_ENTITY', 'SECONDARY_ENTITY_BUSINESS_NAME', 'WORKSITE_STATE'
           , 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY'
           , 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL']
#           , 'PW_OES_YEAR', 'PW_OTHER_SOURCE', 'PW_OTHER_YEAR', 'PW_SURVEY_PUBLISHER', 'PW_SURVEY_NAME']



# Fiscal Year 2023

In [3]:
quarter_first = 1
quarter_last = 4

data_quarter_list = []
for quarter_temp in range(quarter_first, quarter_last +1):
    url_temp = f"https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q{quarter_temp}.xlsx"
    data_quarter = pd.read_excel(url_temp)
#    data_quarter = data_quarter.rename(columns={'Initial Approvals':'Initial Approval',
#                                         'Initial Denials':'Initial Denial',
#                                        'Continuing Approvals':'Continuing Approval',
#                                         'Continuing Denials':'Continuing Denial'})
    data_quarter_list.append(data_quarter)
    


In [4]:
df = pd.concat(data_quarter_list, ignore_index=True)

df = df[(df['VISA_CLASS']=='H-1B') & \
        (df['FULL_TIME_POSITION']=='Y')]

df

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,WILLFUL_VIOLATOR,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL
0,I-200-22356-657078,Certified,2022-12-22,2022-12-30,NaT,H-1B,DevOps Engineer II,15-1252.00,Software Developers,Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
1,I-200-22356-657600,Certified,2022-12-22,2022-12-30,NaT,H-1B,FE Engineer,17-2072.00,"Electronics Engineers, Except Computer",Y,...,No,NaN,NaN,NaN,Disclose Business,NAKAMURA,ANNE,M,"FRAGOMEN, DEL REY, BERNSEN & LOEWY, LLP.",hbal@fragomen.com
2,I-200-22356-657298,Certified,2022-12-22,2022-12-30,NaT,H-1B,Control Sys Engineer,17-2141.00,Mechanical Engineers,Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
3,I-200-22356-657871,Certified,2022-12-22,2022-12-30,NaT,H-1B,Senior Lecturer,25-1021.00,"Computer Science Teachers, Postsecondary",Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
4,I-200-22356-657847,Certified,2022-12-22,2022-12-30,NaT,H-1B,Principal Payment Tech Development,15-1252.00,Software Developers,Y,...,No,NaN,NaN,NaN,Disclose Business,Chegu,Reshma,NaN,"Fragomen, Del Rey, Bernsen & Loewy, LLP",rchegu@fragomen.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644602,I-200-23180-156120,Withdrawn,2023-06-29,2023-07-02,NaT,H-1B,Full Stack Developer,15-1252.00,Software Developers,Y,...,No,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
644603,I-200-23180-156036,Withdrawn,2023-06-29,2023-07-02,NaT,H-1B,Full Stack Developer,15-1252.00,Software Developers,Y,...,No,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
644604,I-200-23179-150073,Withdrawn,2023-06-28,2023-07-02,NaT,H-1B,Software Engineer - Data & Analytics,15-1252.00,Software Developers,Y,...,No,NaN,NaN,NaN,Disclose Business and Employment,NaN,NaN,NaN,NaN,NaN
644605,I-200-21211-494598,Certified - Withdrawn,2021-07-30,2023-07-01,2021-08-06,H-1B,Software Engineer,15-1132.00,"Software Developers, Applications",Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN


In [5]:
min(df['DECISION_DATE'])

Timestamp('2022-10-01 00:00:00')

In [6]:
max(df['DECISION_DATE'])

Timestamp('2023-09-30 00:00:00')

In [7]:
df23 = df

In [8]:
# number of duplicate rows
len(df23)-len(df23.drop_duplicates())

93234

In [9]:
# number of duplicate Case Numbers
len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

96059

In [10]:
# remove duplcate cases, keeping row with latest Decision Date

df23 = df23.sort_values(['CASE_NUMBER', 'DECISION_DATE'], ascending = False) \
    .drop_duplicates(subset = 'CASE_NUMBER', keep = 'first', ignore_index = True)

In [11]:
# number of duplicate Case Numbers
len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

0

In [12]:
#dup = df23[df23.duplicated(['CASE_NUMBER'], keep = False)]\
#     .sort_values(['CASE_NUMBER', 'DECISION_DATE'], ascending = False)

#dup

In [13]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521711 entries, 0 to 521710
Data columns (total 96 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   CASE_NUMBER                     521711 non-null  object        
 1   CASE_STATUS                     521711 non-null  object        
 2   RECEIVED_DATE                   521711 non-null  datetime64[ns]
 3   DECISION_DATE                   521711 non-null  datetime64[ns]
 4   ORIGINAL_CERT_DATE              32008 non-null   datetime64[ns]
 5   VISA_CLASS                      521711 non-null  object        
 6   JOB_TITLE                       521711 non-null  object        
 7   SOC_CODE                        521711 non-null  object        
 8   SOC_TITLE                       521711 non-null  object        
 9   FULL_TIME_POSITION              521711 non-null  object        
 10  BEGIN_DATE                      521711 non-null  datetim

In [14]:
# Add WORKSITE_EMPLOYER column

df23['WORKSITE_EMPLOYER'] = df23['EMPLOYER_NAME']
df23.loc[df23['SECONDARY_ENTITY'] == 'Yes', 'WORKSITE_EMPLOYER'] = df23['SECONDARY_ENTITY_BUSINESS_NAME']

df23[['EMPLOYER_NAME','SECONDARY_ENTITY_BUSINESS_NAME','WORKSITE_EMPLOYER']]

,EMPLOYER_NAME,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_EMPLOYER
0,"Mateu Architecture, Inc.",NaN,"Mateu Architecture, Inc."
1,"SPORTSMED PT, LLC",NaN,"SPORTSMED PT, LLC"
2,"EXA DATA SOLUTIONS, INC.",Walmart,Walmart
3,"EXA DATA SOLUTIONS, INC.",Intuit,Intuit
4,"EXA DATA SOLUTIONS, INC.",Intuit,Intuit
...,...,...,...
521706,"Megan Soft, Inc.",Ford Motor Company,Ford Motor Company
521707,"BETA CAE Systems USA, Inc.",NaN,"BETA CAE Systems USA, Inc."
521708,"MARLABS, INC.",PAYCHEX INC,PAYCHEX INC
521709,"IT Division, Inc.",Kroger,Kroger


In [15]:
# df23['WORKSITE_EMPLOYER'] = df23['WORKSITE_EMPLOYER'].str.upper()
# df23['WORKSITE_EMPLOYER']

In [16]:
# Facebook changes company name to Meta
# https://www.cnbc.com/2021/10/28/facebook-changes-company-name-to-meta.html

#Excel function =CONCAT(", ", "'",A2, "':'",$A$1,"'")



dict_emp = {'Meta Platforms, Inc. ':'Meta Platforms, Inc.'
            , '="Meta Platforms, Inc. "Facebook""':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc':'Meta Platforms, Inc.'
            , 'META Platforms INC':'Meta Platforms, Inc.'
            , 'META PLATFORMS, INC. (FACEBOOK, INC.)':'Meta Platforms, Inc.'
            , 'Meta Platforms':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc. (â€œFacebookâ€)':'Meta Platforms, Inc.'
            , 'Meta Inc':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc.':'Meta Platforms, Inc.'
            , 'Meta Plaforms, Inc.':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc;Â ':'Meta Platforms, Inc.'
            , '="Meta Platforms Inc. ("Facebook")"':'Meta Platforms, Inc.'
            , 'Meta/Facebook':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc., DBA â€œMetaâ€':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc. (META)':'Meta Platforms, Inc.'
            , 'Facebook/Meta':'Meta Platforms, Inc.'
            , 'Meta,':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc. f/k/a Facebook, Inc.':'Meta Platforms, Inc.'
            , 'New Meta Entertainment, Inc.':'Meta Platforms, Inc.'
            , 'Meta (Facebook ,Inc)':'Meta Platforms, Inc.'
            , 'Meta (Facebook)':'Meta Platforms, Inc.'
            , 'HCL-Meta':'Meta Platforms, Inc.'
            , 'Meta (Facebook) (Remote Work Location)':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc. d/b/a Meta':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc.,':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc., â€œFacebookâ€':'Meta Platforms, Inc.'
            , '="Meta Platforms, Inc. ("Facebook")"':'Meta Platforms, Inc.'
            , 'META SOFT INC':'Meta Platforms, Inc.'
            , 'Meta Inc.':'Meta Platforms, Inc.'
            , 'Meta- Managed Service':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc (Facebook Inc)':'Meta Platforms, Inc.'
            , '(Giphy) Meta':'Meta Platforms, Inc.'
            , 'META Platforms Inc Office':'Meta Platforms, Inc.'
            , 'META PLATFORMS, INC. ':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc. (â€œFacebookâ€)':'Meta Platforms, Inc.'
            , 'Meta Payments, Inc.':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc;Â ':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc., â€œFacebookâ€':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc. (Facebook, Inc.)':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc., DBA â€œMetaâ€':'Meta Platforms, Inc.'
            , 'Meta INC':'Meta Platforms, Inc.'
            , 'META PLATFORMS, INC':'Meta Platforms, Inc.'

            , 'Facebook Inc':'Meta Platforms, Inc.'
            , 'Facebook':'Meta Platforms, Inc.'
            , 'Meta Platforms Inc. (â€œFacebookâ€)':'Meta Platforms, Inc.'
            , 'HCL AMERICA / FACEBOOK':'Meta Platforms, Inc.'
            , 'FACEBOOK , INC':'Meta Platforms, Inc.'
            , 'Facebook Technologies, LLC':'Meta Platforms, Inc.'
            , 'Facebook, Inc':'Meta Platforms, Inc.'
            , 'Quest Global Services / Facebook':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc., â€œFacebookâ€':'Meta Platforms, Inc.'
            , 'Meta Platforms, Inc. (Facebook, Inc.)':'Meta Platforms, Inc.'
            , 'Facebook, Inc.':'Meta Platforms, Inc.'

            
            , 'AMAZON WEB SERVICES, INC.':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DEVELOPMENT CENTER U.S., INC.':'AMAZON.COM SERVICES LLC'
            , 'Remote Work From Home for Amazon':'AMAZON.COM SERVICES LLC'
            , 'Remote work From Home for Amazon':'AMAZON.COM SERVICES LLC'
            , 'Amazon Data Services, Inc.':'AMAZON.COM SERVICES LLC'
            , 'AMAZON':'AMAZON.COM SERVICES LLC'
            , 'AMAZON ADVERTISING LLC':'AMAZON.COM SERVICES LLC'
            , 'AMAZON.COM, INC.':'AMAZON.COM SERVICES LLC'
            , 'Work From home for Amazon':'AMAZON.COM SERVICES LLC'
            , 'Amazon com Inc':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com Services Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com Inc':'AMAZON.COM SERVICES LLC'
            , 'AMAZON (REMOTE WORK)':'AMAZON.COM SERVICES LLC'
            , 'Amazon Web Services Inc. ':'AMAZON.COM SERVICES LLC'
            , 'AMAZON CAPITAL SERVICES, INC.':'AMAZON.COM SERVICES LLC'
            , 'Persistent/Amazon':'AMAZON.COM SERVICES LLC'
            , 'AMAZON (TELECOMMUTING)':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DEVELOPMENT CENTER U.S. INC.':'AMAZON.COM SERVICES LLC'
            , 'Remote Worksite (Amazon)':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com':'AMAZON.COM SERVICES LLC'
            , 'Amazon, Inc. ':'AMAZON.COM SERVICES LLC'
            , 'Amazon Web Services, Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Development Center U.S., Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Data Services, Inc':'AMAZON.COM SERVICES LLC'
            , 'Amazon Advertising LLC':'AMAZON.COM SERVICES LLC'
            , 'Amazon Web Services Inc':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DATA SERVICES, INC.':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com.ca, Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Media Venture LLC':'AMAZON.COM SERVICES LLC'
            , 'Amazon Payments, Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DATA SERVICES, INC. ':'AMAZON.COM SERVICES LLC'
            , 'Amazon Capital Services, Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Studios, LLC':'AMAZON.COM SERVICES LLC'
            , 'Remote work from Home for Amazon':'AMAZON.COM SERVICES LLC'
            , 'Remote Work from Home for Amazon':'AMAZON.COM SERVICES LLC'
            , 'Amazon Development Center U.S. Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Mechanical Turk, Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Robotics LLC':'AMAZON.COM SERVICES LLC'
            , 'Amazon Web Services Inc.':'AMAZON.COM SERVICES LLC'
            , 'Amazon Retail LLC ':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DEVELOPMENT CENTER U.S., INC. ':'AMAZON.COM SERVICES LLC'
            , 'Amazon Media Venture LLC ':'AMAZON.COM SERVICES LLC'
            , 'AMAZON CAPITAL SERVICES, INC.':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com, Inc.':'AMAZON.COM SERVICES LLC'
            , 'AMAZON DEVELOPMENT CENTR U.S., INC.':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com.ca, Inc. (US)':'AMAZON.COM SERVICES LLC'
            , 'Work from home for amazon':'AMAZON.COM SERVICES LLC'
            , 'Amazon Web Services, Inc. ':'AMAZON.COM SERVICES LLC'
            , 'Amazon Fresh LLC':'AMAZON.COM SERVICES LLC'
            , 'Amazon.com Services LLC':'AMAZON.COM SERVICES LLC'

            , 'Qualcomm':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Inc.':'Qualcomm Technologies, Inc.'
            , 'Qualcomm, Inc. ':'Qualcomm Technologies, Inc.'
            , 'Qualcomm ':'Qualcomm Technologies, Inc.'
            , 'Arriver/Qualcomm':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Technologies Inc.':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Atheros, Inc. ':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Innovation Center, Inc. ':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Incorporated':'Qualcomm Technologies, Inc.'
            , 'Qualcomm Innovation Center, Inc.':'Qualcomm Technologies, Inc.'

            , 'CISCO Systems Inc':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS INC.':'Cisco Systems, Inc.'
            , 'Cisco':'Cisco Systems, Inc.'
            , 'Cisco Systems, Inc':'Cisco Systems, Inc.'
            , 'CISCO Systems':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc ':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS INC.,':'Cisco Systems, Inc.'
            , 'Cisco System Inc':'Cisco Systems, Inc.'
            , 'Cisco Systems, Inc. ':'Cisco Systems, Inc.'
            , 'Cisco System Inc.':'Cisco Systems, Inc.'
            , 'Cisco Systems, Inc  ':'Cisco Systems, Inc.'
            , 'CISCO ':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc. ':'Cisco Systems, Inc.'
            , 'TCS/Cisco':'Cisco Systems, Inc.'
            , 'Cisco Systems (Remote Working)':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc, USA':'Cisco Systems, Inc.'
            , 'REMOTE LOCATION - (CISCO)':'Cisco Systems, Inc.'
            , 'Cisco Systems, Inc. (Working Remotely)':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS INC (REMOTE)':'Cisco Systems, Inc.'
            , 'Cisco Systems, Inc.(Remote Work Location)':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc / Remote':'Cisco Systems, Inc.'
            , 'Remote Location (CISCO)':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc / Remote ':'Cisco Systems, Inc.'
            , '(Remote) Cisco Systems, Inc':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS INC':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc.':'Cisco Systems, Inc.'
            , 'CISCO':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS, INC':'Cisco Systems, Inc.'
            , 'Cisco Systems':'Cisco Systems, Inc.'
            , 'Cisco Systems Inc.,':'Cisco Systems, Inc.'
            , 'CISCO System Inc.':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS INC':'Cisco Systems, Inc.'
            , 'CISCO SYSTEMS':'Cisco Systems, Inc.'
            , 'CISCO Systems Inc.':'Cisco Systems, Inc.'
            , 'Cisco systems':'Cisco Systems, Inc.'

            , 'Ernst & Young':'Ernst & Young U.S. LLP'
            , 'Ernst &  Young':'Ernst & Young U.S. LLP'
            , 'TCS / ERNST & YOUNG':'Ernst & Young U.S. LLP'
            , 'Ernst &Young':'Ernst & Young U.S. LLP'
            , 'Ernst And Young - American Family Insurance':'Ernst & Young U.S. LLP'
            , 'Ernst and Young LLC':'Ernst & Young U.S. LLP'
            , 'Ernst & Young / Fortive Corporation':'Ernst & Young U.S. LLP'
            , '(Remote) Ernst & Young':'Ernst & Young U.S. LLP'
            , 'Ernst & Young L.L.P. ':'Ernst & Young U.S. LLP'
            , 'Ernst & Young LLP (EY GUAM)':'Ernst & Young U.S. LLP'
            , 'ERNST & YOUNG':'Ernst & Young U.S. LLP'

            , 'GOOGLE':'Google LLC'
            , 'Google Inc':'Google LLC'
            , 'Google Inc.':'Google LLC'
            , 'Google Public Sector LLC ':'Google LLC'
            , 'Google ':'Google LLC'
            , 'Google, Inc':'Google LLC'
            , 'Google, LLC':'Google LLC'
            , 'Google LLC ':'Google LLC'
            , 'Google (Remote)':'Google LLC'
            , 'Google, Inc.':'Google LLC'
            , 'GOOGLE LLC.':'Google LLC'
            , ' Google Inc':'Google LLC'
            , 'FOR GOOGLE':'Google LLC'
            , 'Remote work from Residence for Google':'Google LLC'
            , 'Google, Inc. (Remote Home Office)':'Google LLC'
            , 'Google (Remote Working)':'Google LLC'
            , 'Google, Inc. (Working Remotely)':'Google LLC'
            , 'HCL America, Inc./Google LLC':'Google LLC'
            , 'Google Fiber':'Google LLC'
            , 'Google Public Sector LLC':'Google LLC'
            , 'HCL/ Google':'Google LLC'
            , 'Google ,':'Google LLC'
            , 'Working Remotely for Google':'Google LLC'
            , 'Google(Remote)':'Google LLC'
            , 'Google Austin':'Google LLC'
            , 'Google':'Google LLC'
            , 'GOOGLE INC':'Google LLC'
            , 'GOOGLE Inc.':'Google LLC'
            , 'Google LLC.':'Google LLC'
            
            , 'Oracle America, Inc':'Oracle America, Inc.'
            , 'ORACLE FINANCIAL SERVICES SOFTWARE, INC.':'Oracle America, Inc.'
            , 'Oracle America, Inc. ':'Oracle America, Inc.'
            , 'Telecommuter to Oracle Corporation ':'Oracle America, Inc.'
            , 'Telecommuter to Oracle Corporation':'Oracle America, Inc.'
            , 'Oracle America Inc':'Oracle America, Inc.'
            , 'ORACLE':'Oracle America, Inc.'
            
            , 'McKinsey & Company, Inc':'McKinsey & Company, Inc. United States'
            , 'MCKINSEY & COMPANY':'McKinsey & Company, Inc. United States'
            , 'McKinsey and Company, Inc.':'McKinsey & Company, Inc. United States'

            , 'Apple, Inc.':'Apple Inc.'
            , 'Apple Inc':'Apple Inc.'
            , 'APPLE':'Apple Inc.'
            , 'Apple, Inc. ':'Apple Inc.'
            , 'APPLE PROCESSING  LLC':'Apple Inc.'
            , 'Apple, Inc':'Apple Inc.'
            , 'TCS/APPLE':'Apple Inc.'
            , 'Apple ':'Apple Inc.'
            , 'APPLE PROCESSING LLC':'Apple Inc.'
            , 'Apple,  Inc.':'Apple Inc.'
            , 'Apple American Group LLC':'Apple Inc.'
            , 'Apple Computer Inc':'Apple Inc.'
            , 'APPLE INC (TELECOMMUTING)':'Apple Inc.'
            , 'APPLE INC ':'Apple Inc.'
            , 'Apple Financing LLC':'Apple Inc.'
            , 'Apple Corporation':'Apple Inc.'
            , 'Infosys / Apple ':'Apple Inc.'
            , 'Remote Worksite (Apple Inc.) ':'Apple Inc.'
            , 'Infosys/Apple Inc.':'Apple Inc.'
            , 'TCS / Apple Inc':'Apple Inc.'
            , 'Apple .':'Apple Inc.'
            , '(Remote) Apple Computer Inc':'Apple Inc.'
            , 'Apple, Inc. (Vendor\'s location)':'Apple Inc.'
            , 'Â Apple, Inc':'Apple Inc.'
            , 'Apple/TCS':'Apple Inc.'
            , 'APPLE - AGILEONE':'Apple Inc.'
            , 'Apple Inc / Remote':'Apple Inc.'
            , 'Apple, IncÂ ':'Apple Inc.'
            , 'Apple Inc,':'Apple Inc.'
            , 'Cognizant/ Apple':'Apple Inc.'
            , 'TCS/ APPLE':'Apple Inc.'
            , 'InfoSys/Apple':'Apple Inc.'
            , ' Apple, Inc. ':'Apple Inc.'
            , 'Remote Worksite (Apple Inc.)':'Apple Inc.'
            , 'TCS / Apple':'Apple Inc.'
            , 'Apple Inc.,':'Apple Inc.'
            , 'APPLE (WORK FROM HOME)':'Apple Inc.'
            , 'Â Apple, Inc(Remote)':'Apple Inc.'
            , ' Apple':'Apple Inc.'
            , 'Apple Inc. ':'Apple Inc.'

            , 'Microsoft':'Microsoft Corporation'
            , 'Remote Work from Home for Microsoft':'Microsoft Corporation'
            , 'Microsoft Corporation ':'Microsoft Corporation'
            , 'Microsoft Inc':'Microsoft Corporation'
            , 'Microsoft Corporation(Remote Work Location)':'Microsoft Corporation'
            , 'MICROSOFT - REMOTE':'Microsoft Corporation'
            , 'Microsoft Technology Center':'Microsoft Corporation'
            , 'Microsoft (Beneficiary\'s Residence - Remote Work Location) ':'Microsoft Corporation'
            , 'TCS/Microsoft (Remote work)':'Microsoft Corporation'
            , 'Microsoft (Remote)':'Microsoft Corporation'
            , 'MICROSOFT ( REMOTE WORK)':'Microsoft Corporation'
            , 'Microsoft Corportation':'Microsoft Corporation'
            , 'Microsoft ':'Microsoft Corporation'
            , 'MicroSoft/ Remote':'Microsoft Corporation'
            , ' Microsoft Corporation':'Microsoft Corporation'
            , 'TCS/Microsoft - Remote':'Microsoft Corporation'
            , 'Microsoft Corporation (Working Remotely)':'Microsoft Corporation'
            , 'Microsoft ( Remote )':'Microsoft Corporation'
            , 'MICROSOFT':'Microsoft Corporation'
            , 'Remote Work From Home for Microsoft':'Microsoft Corporation'

            , 'Intel Corporation':'Intel Corporation '
            , 'Intel Americas, Inc. ':'Intel Corporation '
            , 'Intel':'Intel Corporation '
            , 'INTEL ':'Intel Corporation '
            , 'Intel Americas':'Intel Corporation '

            , 'Cognizant TriZetto Software Group, Inc.':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Trizetto Corporation - A Cognizant Company':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'COGNIZANT':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions_IQN/ TD Bank':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions US Corporation':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Mobility, Inc.':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'TMG HEALTH - A COGNIZANT COMPANY':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'TIAA CREF / COGNIZANT (REMOTE)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Anthem Inc. and Cognizant Technology Solutions US Corp.':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant ':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions/CVS Health':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions (CTS) ':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant/ Apple':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions [Project for PepsiCo (Remotely working from home)]':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions(Project for Ford Motor Company)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant / Liberty Mutual Insurance':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solution [Project for Centene Corporation] [Working Remotely from Home Office]':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'TD Bank/ Cognizant':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'COGNIZANT WORLDWIDE LIMITED':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant / American Express':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant/BSC':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technologies Serives':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Services (REMOTE / WFH)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'AIG / COGNIZANT':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions project for CVS Pharmacy,Inc. (Remotely working from home office)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions project for Columbia Sportswear Company(Remotely working from home Office)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'
            , 'Cognizant Technology Solutions project for CVS Pharmacy, Inc. (Remotely working from home office)':'COGNIZANT TECHNOLOGY SOLUTIONS US CORP'

            , 'Infosys':'INFOSYS LIMITED'
            , 'Srivin Infosystems, Inc.':'INFOSYS LIMITED'
            , 'INFOSYS BPM LIMITED':'INFOSYS LIMITED'
            , 'INFOSYS MCCAMISH SYSTEMS, LLC ':'INFOSYS LIMITED'
            , 'CHARTER COMMUNICATIONS / INFOSYS':'INFOSYS LIMITED'
            , 'Vectra Infosys, Inc.':'INFOSYS LIMITED'
            , 'INFOSYS MCCAMISH SYSTEMS, LLC':'INFOSYS LIMITED'
            , 'BRENNTAG / INFOSYS (TELECOMMUTING)':'INFOSYS LIMITED'
            , 'Infosys/Molina Healthcare':'INFOSYS LIMITED'
            , 'Infosys [Project for Premium Brands] (Remotely working from home)':'INFOSYS LIMITED'
            , 'STERLING INFOSYSTEMS, INC.':'INFOSYS LIMITED'
            , 'PROSPECT INFOSYSTEM, INC.':'INFOSYS LIMITED'
            , 'Infosys/UBS':'INFOSYS LIMITED'
            , 'Remotely working for Infosys':'INFOSYS LIMITED'
            , 'Infosys Ltd':'INFOSYS LIMITED'
            , 'NOUS INFOSYSTEMS PVT. LTD.':'INFOSYS LIMITED'
            , 'INFOSYS McCAMISH SYSTEMS LLC':'INFOSYS LIMITED'
            , 'Rave Infosys, Inc.':'INFOSYS LIMITED'
            , 'Infosys / BP North America Inc':'INFOSYS LIMITED'
            , 'Infosys/T-Mobile':'INFOSYS LIMITED'
            , 'INFOSYS ( REMOTE )':'INFOSYS LIMITED'
            , 'Infosys/Walmart':'INFOSYS LIMITED'
            , 'BP/ Infosys Pontoon':'INFOSYS LIMITED'
            , 'Peloton Interactive/ Infosys':'INFOSYS LIMITED'
            , 'Direct TV / Infosys (Implementation Partner)':'INFOSYS LIMITED'
            , 'Prospect Infosystem, Inc. ':'INFOSYS LIMITED'
            , 'GAP, INC THROUGH INFOSYS':'INFOSYS LIMITED'
            , 'Remote work from Residence for Infosys/Hallmark':'INFOSYS LIMITED'
            , 'INFOSYS/FISERV':'INFOSYS LIMITED'
            , 'INFOSYS (TELECOMMUTING)':'INFOSYS LIMITED'
            , 'CGINFOSYS Inc':'INFOSYS LIMITED'
            , 'Infosys Public Systems (Telecommuting From)':'INFOSYS LIMITED'
            , 'Infosys/ United Airlines':'INFOSYS LIMITED'
            , 'Infosys Project for Atlice USA':'INFOSYS LIMITED'

           }

# df23.WORKSITE_EMPLOYER = df23.WORKSITE_EMPLOYER.replace(dict_emp)  
df23['WORKSITE_EMPLOYER'] = df23['WORKSITE_EMPLOYER'].replace(dict_emp)


In [17]:

# Fill in missing values with an empty string
df23['WORKSITE_EMPLOYER'] = df23['WORKSITE_EMPLOYER'].fillna('')

# Replace values in the 'WORKSITE_EMPLOYER' column based on condition contains('FACEBOOK')
# mask = df23['WORKSITE_EMPLOYER'].str.contains('FACEBOOK')
# df23.loc[mask, 'WORKSITE_EMPLOYER'] = 'META PLATFORMS INC'

In [18]:
df23

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL,WORKSITE_EMPLOYER
0,I-200-23273-403071,Withdrawn,2023-09-30,2023-09-30,NaT,H-1B,Architectural Designer,17-1011.00,"Architects, Except Landscape and Naval",Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,"Mateu Architecture, Inc."
1,I-200-23273-401613,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Physical Therapist,29-1123.00,Physical Therapists,Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NOT APPLICABLE,NaN,"SPORTSMED PT, LLC"
2,I-200-23272-399379,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Data Scientist,15-1299.08,Computer Systems Engineers/Architects,Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Walmart
3,I-200-23272-399363,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Java Developer,15-1252.00,Software Developers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Intuit
4,I-200-23272-399344,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Java Developer,15-1251.00,Computer Programmers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Intuit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521706,I-200-19276-070288,Certified - Withdrawn,2019-10-03,2022-11-21,2019-10-10,H-1B,Sr. Software Developer,15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,Hope,Shawn,T.,"Siegel, Gross & Tou, P.C.",shawn@sgimmigration.com,Ford Motor Company
521707,I-200-19276-070268,Certified - Withdrawn,2019-10-03,2023-01-05,2019-10-10,H-1B,FEA Engineer,17-2141.00,Mechanical Engineers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,Maroko,Eli,NaN,"Jaffe Raitt Heuer & Weiss, P.C.",jwhite@jaffelaw.com,"BETA CAE Systems USA, Inc."
521708,I-200-19276-069830,Certified - Withdrawn,2019-10-03,2022-10-11,2019-10-10,H-1B,Software Application Developer,15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,COMPLIDO,MARILOU,S,"Marlabs, Inc",MARILOU@MARLABS.COM,PAYCHEX INC
521709,I-200-19276-069560,Certified - Withdrawn,2019-10-03,2023-01-21,2019-10-10,H-1B,IT Consultant (JAVA Developer),15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,Sanoussi,Sabirou,T,"Mannam & Associates, LLC",lawxpert@yahoo.com,Kroger


In [19]:
#df23['WAGE_UNIT_OF_PAY'].unique()


In [20]:
#df23['PW_UNIT_OF_PAY'].unique()

In [21]:
# Add WAGE_RATE_OF_PAY_FROM_YR column

# df23['WAGE_RATE_OF_PAY_FROM_YR'] = df23['WAGE_RATE_OF_PAY_FROM']
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Hour', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 2000
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Month', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 12
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Week', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 50
# df23.loc[df23['WAGE_UNIT_OF_PAY'] == 'Bi-Weekly', 'WAGE_RATE_OF_PAY_FROM_YR'] = df23.WAGE_RATE_OF_PAY_FROM * 25


# # Add PREVAILING_WAGE_YR column

# df23['PREVAILING_WAGE_YR'] = df23['PREVAILING_WAGE']
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Hour', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 2000
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Month', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 12
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Week', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 50
# df23.loc[df23['PW_UNIT_OF_PAY'] == 'Bi-Weekly', 'PREVAILING_WAGE_YR'] = df23.PREVAILING_WAGE * 25

# # Add WAGE_DIFF column

# df23['WAGE_DIFF'] = (df23.WAGE_RATE_OF_PAY_FROM_YR- df23.PREVAILING_WAGE_YR)/df23.PREVAILING_WAGE_YR

# df23[['WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY'
#       , 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY'
#       , 'WAGE_RATE_OF_PAY_FROM_YR', 'PREVAILING_WAGE_YR', 'WAGE_DIFF']]


In [22]:
# df23['WAGE_DIFF'].max()

In [23]:
# df23['WAGE_DIFF'].min()

In [24]:
# len(df23[df23['WAGE_DIFF'] < 0])

In [25]:
rslt_df = df23[df23['WAGE_UNIT_OF_PAY'] == 'Year'] 
rslt_df

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL,WORKSITE_EMPLOYER
0,I-200-23273-403071,Withdrawn,2023-09-30,2023-09-30,NaT,H-1B,Architectural Designer,17-1011.00,"Architects, Except Landscape and Naval",Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,"Mateu Architecture, Inc."
1,I-200-23273-401613,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Physical Therapist,29-1123.00,Physical Therapists,Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NOT APPLICABLE,NaN,"SPORTSMED PT, LLC"
5,I-200-23272-399322,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Software Engineer,15-1252.00,Software Developers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,I2U SYSTEMS INC
6,I-200-23272-399218,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Statistical Research Associate IV,15-2041.00,Statisticians,Y,...,NaN,NaN,NaN,Disclose Business,Garrett-Cowan,Amy,NaN,Fred Hutchinson Cancer Center,acowan@fredhutch.org,Fred Hutchinson Cancer Center
8,I-200-23272-396955,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,VMWARE Solutions Specialist,15-1299.08,Computer Systems Engineers/Architects,Y,...,NaN,NaN,NaN,Disclose Business,Harding,Richard,A,"Global Immigration Partners, Inc",Admin@gip-us.com,AIN LLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521704,I-200-19277-071859,Certified - Withdrawn,2019-10-04,2022-12-09,2019-10-11,H-1B,Product Design Engineer,17-2141.00,Mechanical Engineers,Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Mando America Corporation
521705,I-200-19276-070645,Certified - Withdrawn,2019-10-03,2022-11-22,2019-10-10,H-1B,Assistant Professor,25-1031.00,"Architecture Teachers, Postsecondary",Y,...,NaN,NaN,NaN,Disclose Business,Leinwohl,Lynn,C,"Arizona State University, International Scholars",lynn.leinwohl@asu.edu,Arizona State University
521707,I-200-19276-070268,Certified - Withdrawn,2019-10-03,2023-01-05,2019-10-10,H-1B,FEA Engineer,17-2141.00,Mechanical Engineers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,Maroko,Eli,NaN,"Jaffe Raitt Heuer & Weiss, P.C.",jwhite@jaffelaw.com,"BETA CAE Systems USA, Inc."
521708,I-200-19276-069830,Certified - Withdrawn,2019-10-03,2022-10-11,2019-10-10,H-1B,Software Application Developer,15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,COMPLIDO,MARILOU,S,"Marlabs, Inc",MARILOU@MARLABS.COM,PAYCHEX INC


In [26]:
# len(rslt_df[rslt_df['WAGE_DIFF'] < 0])

In [27]:
# rslt_df = rslt_df['WAGE_DIFF']
# rslt_df.hist(bins = 1000)

In [28]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521711 entries, 0 to 521710
Data columns (total 97 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   CASE_NUMBER                     521711 non-null  object        
 1   CASE_STATUS                     521711 non-null  object        
 2   RECEIVED_DATE                   521711 non-null  datetime64[ns]
 3   DECISION_DATE                   521711 non-null  datetime64[ns]
 4   ORIGINAL_CERT_DATE              32008 non-null   datetime64[ns]
 5   VISA_CLASS                      521711 non-null  object        
 6   JOB_TITLE                       521711 non-null  object        
 7   SOC_CODE                        521711 non-null  object        
 8   SOC_TITLE                       521711 non-null  object        
 9   FULL_TIME_POSITION              521711 non-null  object        
 10  BEGIN_DATE                      521711 non-null  datetim

In [29]:
df23['PW_WAGE_LEVEL'].unique()

array(['I', 'IV', 'II', 'III', nan], dtype=object)

In [30]:
min(df23['DECISION_DATE'])

Timestamp('2022-10-01 00:00:00')

In [31]:
max(df23['DECISION_DATE'])

Timestamp('2023-09-30 00:00:00')

In [32]:
len(df23)-len(df23.drop_duplicates())

0

In [33]:
len(df23['CASE_NUMBER'])-len(df23['CASE_NUMBER'].drop_duplicates())

0

In [34]:
# Export df to compressed .csv

compression_opts = dict(method='zip', archive_name='h1b_lca_fy23.csv')  

df23.to_csv('h1b_lca_fy23_data.zip', index=False, compression=compression_opts)  

In [35]:
df23

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL,WORKSITE_EMPLOYER
0,I-200-23273-403071,Withdrawn,2023-09-30,2023-09-30,NaT,H-1B,Architectural Designer,17-1011.00,"Architects, Except Landscape and Naval",Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,"Mateu Architecture, Inc."
1,I-200-23273-401613,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Physical Therapist,29-1123.00,Physical Therapists,Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NOT APPLICABLE,NaN,"SPORTSMED PT, LLC"
2,I-200-23272-399379,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Data Scientist,15-1299.08,Computer Systems Engineers/Architects,Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Walmart
3,I-200-23272-399363,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Java Developer,15-1252.00,Software Developers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Intuit
4,I-200-23272-399344,Withdrawn,2023-09-29,2023-09-29,NaT,H-1B,Java Developer,15-1251.00,Computer Programmers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,Intuit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521706,I-200-19276-070288,Certified - Withdrawn,2019-10-03,2022-11-21,2019-10-10,H-1B,Sr. Software Developer,15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,Hope,Shawn,T.,"Siegel, Gross & Tou, P.C.",shawn@sgimmigration.com,Ford Motor Company
521707,I-200-19276-070268,Certified - Withdrawn,2019-10-03,2023-01-05,2019-10-10,H-1B,FEA Engineer,17-2141.00,Mechanical Engineers,Y,...,Yes,"Both $60,000 or higher in annual wage and Mast...",NaN,Disclose Business,Maroko,Eli,NaN,"Jaffe Raitt Heuer & Weiss, P.C.",jwhite@jaffelaw.com,"BETA CAE Systems USA, Inc."
521708,I-200-19276-069830,Certified - Withdrawn,2019-10-03,2022-10-11,2019-10-10,H-1B,Software Application Developer,15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,COMPLIDO,MARILOU,S,"Marlabs, Inc",MARILOU@MARLABS.COM,PAYCHEX INC
521709,I-200-19276-069560,Certified - Withdrawn,2019-10-03,2023-01-21,2019-10-10,H-1B,IT Consultant (JAVA Developer),15-1132.00,"Software Developers, Applications",Y,...,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,Sanoussi,Sabirou,T,"Mannam & Associates, LLC",lawxpert@yahoo.com,Kroger
